<a href="https://colab.research.google.com/github/ciaoCiao100/proveChallenge2/blob/master/customerTensorflowProva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

vengono importate le librerie , scaricato il dataset e impostata la target label

In [37]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# operazione che fa in modo che i numeri siano più leggibili
np.set_printoptions(precision=3, suppress=True)

data_path ="/content/drive/My Drive/WA_Fn-UseC_-Telco-Customer-Churn.train.csv"
df = pd.read_csv(data_path)
target_label ="Churn"
df.head()


,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1,No,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
2,Yes,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
3,Yes,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
4,No,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,


controllo di valori non validi

In [38]:
# Sostituisco ad ogni stringa vuola il valore 'NaN'
df.replace("", np.nan, inplace=True)
df.replace(" ", np.nan, inplace=True)
df.replace("?", np.nan, inplace=True)
df.head()

,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1,No,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
2,Yes,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
3,Yes,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
4,No,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN


elimino righe con NaN

In [41]:

# Procediamo con l'eliminazione dei dati non validi (NaN)
df.dropna(inplace=True)
df.head()

,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1,No,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
2,Yes,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
3,Yes,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
5,No,4291-SHSBH,Male,0,No,No,7,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,69.55,521.35


conversione degli elementi presenti in tabella in interi 

In [42]:
#tutte le colonne che contengono stringhe devono essere trasformate in colonne che contengono interi 
#for elemento in df:
 
   #df[elemento] = pd.Categorical(df[elemento])
   #df[elemento] = getattr(df, elemento).cat.codes
   #mostro il dataset dopo l'operazione
#df.head()

for elemento in df:
  if elemento=="MonthlyCharges" or elemento=="TotalCharges" or elemento=="tenure":
   df[elemento] = df[elemento].astype('float')
  else:
   df[elemento] = (pd.Categorical(df[elemento])).codes.astype('float')
# mostro il dataset dopo l'operazione
df.head()


,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0.0,2364.0,0.0,0.0,0.0,1.0,21.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,3.0,64.85,1336.80
1,0.0,3448.0,0.0,0.0,0.0,0.0,54.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,97.20,5129.45
2,1.0,2143.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,23.45,23.45
3,1.0,1424.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,70.20,237.95
5,0.0,2394.0,1.0,0.0,0.0,0.0,7.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,69.55,521.35


preparazione del modello della rete

In [43]:
#costruisco il modello
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_compiled_model()
#preparo il dataset
train, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)

addestramento

In [44]:
# Creazione  del training
train_target = train.pop(target_label)
train_dataset = (tf.data.Dataset.from_tensor_slices((train.values, train_target.values))).shuffle(len(train)).batch(1)
#addestramento e impostazione epoche
model.fit(train_dataset, epochs=30)

Epoch 1/30

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

4554/4554 [==============================] - 6s 1ms/step - loss: 12.7674 - accuracy: 0.6836
Epoch 2/30
4554/4554 [==============================] - 6s 1ms/step - loss: 0.9631 - accuracy: 0.7319
Epoch 3/30
4554/4554 [==============================] - 6s 1ms/step - loss: 0.5823 - accuracy: 0.7400
Epoch 4/30
4554/4554 [==============================] - 6s 1ms/step - loss: 0.5332 - accuracy: 0.7510
Epoch 5/30
4554/4554 [==============================] - 6s 1ms/step - loss: 0.5064 - accuracy: 0.75

test

In [45]:
# Creazione del  test 
test_target = test.pop(target_label)
test_dataset = (tf.data.Dataset.from_tensor_slices((test.values, test_target.values))).shuffle(len(test)).batch(1)

# Fase di test
test_loss, test_accuracy = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

563/563 [==============================] - 1s 964us/step - loss: 0.4613 - accuracy: 0.7709


Test Loss 0.46128252148628235, Test Accuracy 0.7708703279495239


validazione

In [46]:
#valutazione
val_target = val.pop(target_label)

val_dataset = (tf.data.Dataset.from_tensor_slices(val.values)).batch(1)

# Fase di valutazione
predictions = model.predict(val_dataset)


# Show some results 
for prediction, yes in zip(predictions[:10], list(val_target)[:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted yes: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("yes" if bool(yes) else "no"))


Predicted yes: 22.80%  | Actual outcome:  no
Predicted yes: 13.27%  | Actual outcome:  no
Predicted yes: 77.27%  | Actual outcome:  yes
Predicted yes: 13.27%  | Actual outcome:  no
Predicted yes: 65.58%  | Actual outcome:  yes
Predicted yes: 21.99%  | Actual outcome:  no
Predicted yes: 9.37%  | Actual outcome:  no
Predicted yes: 79.08%  | Actual outcome:  yes
Predicted yes: 10.94%  | Actual outcome:  no
Predicted yes: 54.33%  | Actual outcome:  yes
